In [73]:
import re
import pandas as pd
from sklearn import metrics

import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
%pylab inline
aspirationList = ['dream','buy', 'favorite','purchase','goal', 'target', 'hope','aim','ideal','ambition', 'desire','wish','want','eager','love']

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [74]:
brand = ['acura', 'audi','bmw', 'buick','cadillac', 'chevrolet', 'chrysler', 'dodge','ford','honda',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'saturn',
 'subaru',
 'suzuki',
 'toyota',
 'volkswagen',
 'volvo',
 'lexus',
 'jaguar']

In [75]:
def dictReplace(x):
    for i in aspirationList:
        my_regex = r"\b" + re.escape(i) + r"\b"
        if re.search(my_regex, x, re.IGNORECASE):
            x = x.replace(i,'aspiration')
    return x

In [76]:
edmunds = pd.read_csv("after_mapping3.csv",sep="\t",encoding='utf-8')
edmunds['text'] = edmunds['text'].str.lower()
edmunds['text'] = edmunds['text'].map(dictReplace)

In [77]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

comment2 = edmunds["text"].tolist()

comment = " ".join(comment2)

In [84]:
word_tokenize = nltk.word_tokenize(comment)

word_tokenize = [w for w in word_tokenize if w.isalpha()]

no_stop = filter(lambda x: x not in stop_words, word_tokenize)

no_stop[:5]

[u'one', u'beauty', u'bmw', u'bmw', u'many']

In [79]:
cleaned_freq = nltk.FreqDist(no_stop)

for word, frequency in cleaned_freq.most_common(30):
    print('%s;%d' % (word, frequency)).encode('utf-8')


car;4487
bmw;4360
acura;3157
aspiration;2305
cars;2166
like;2136
would;2131
infiniti;2015
one;1964
audi;1888
get;1622
think;1454
new;1301
drive;1239
better;1212
much;1175
lexus;1147
good;1082
even;1075
well;994
also;979
driving;970
really;945
performance;924
mercedes;921
people;909
luxury;870
cadillac;869
know;868
price;807


In [80]:
from collections import defaultdict
c = defaultdict(dict)
# make a copy dictionary to handle the case of term pair only count once for each post 
copy = defaultdict(dict)

# insert default value 0 into array
for i in brand:  
    c[i] = 0
# insert default value 0 into array
for z in brand:
    
    copy[i] = 0

In [81]:
brand_freq = {}
aspir_count = 0
for word, frequency in cleaned_freq.most_common(300):
    if word in brand:
        brand_freq[word] = frequency
    if word == 'aspiration':
        aspir_count = frequency
aspir_count

2305

In [82]:
window_size = 10
for sentence in comment2:
    sentence=sentence.strip()
    tokenized = nltk.word_tokenize(sentence)
    tokenized = [w for w in tokenized if w.isalpha()]
    no_stop = filter(lambda x: x not in stop_words, tokenized)
    tokens=[token for token in no_stop if token!=u""]
    for pos,token in enumerate(tokens):
        if token.lower() in brand:
            start=max(0,pos-window_size)
            end=min(len(tokens),pos+window_size+1) 
            aspire = tokens[start:end]
            if 'aspiration' in aspire:
                if c[token] == copy[token]:
                    c[token] = c[token] + 1
                        
           
    copy = c.copy()
c

defaultdict(dict,
            {'acura': 258,
             'audi': 128,
             'bmw': 406,
             'buick': 9,
             'cadillac': 75,
             'chevrolet': 15,
             'chrysler': 5,
             'dodge': 5,
             'ford': 30,
             'honda': 73,
             'hyundai': 22,
             'infiniti': 190,
             'jaguar': 17,
             'kia': 2,
             'lexus': 123,
             'lincoln': 8,
             'mazda': 15,
             'mercedes': 78,
             'mercury': 0,
             'mitsubishi': 3,
             'nissan': 37,
             'pontiac': 12,
             'saturn': 1,
             'subaru': 26,
             'suzuki': 1,
             'toyota': 65,
             'volkswagen': 22,
             'volvo': 35})

In [85]:
len(no_stop)

268971

In [86]:
lift = {}
for i in brand:
    try:    
        lift[i] = (268971*c[i]*1.0)/(aspir_count*cleaned_freq[i]*1.0)
    except:
        print i
    print i,lift[i]

acura 9.53629444467
audi 7.91120261774
bmw 10.8661093753
buick 8.67943923558
cadillac 10.0710792818
chevrolet 11.4402194717
chrysler 6.34186079411
dodge 9.5647736567
ford 13.2602543877
honda 10.9490840857
hyundai 11.4606484351
infiniti 11.0030497947
kia 8.04760266288
lincoln 6.14159150588
mazda 8.1792223326
mercedes 9.88256092477
mercury 0.0
mitsubishi 14.0028286334
nissan 9.10873170598
pontiac 15.2204659059
saturn 14.5862798265
subaru 13.1339662507
suzuki 5.07348863529
toyota 13.7906645632
volkswagen 8.47255857907
volvo 10.0843416084
lexus 12.5134257622
jaguar 4.54984875321
